In [1]:
import torch
import os
import pandas as pd
import scipy
import torchvision
from torch import optim,nn
from torch.utils.data import Dataset
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn.functional as F
from utils import StanfordCars

c:\Users\louis\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
test_dataset = StanfordCars(root=data_root,split ="train",transform=transform)

model = load()

dataiter = iter(test_loader)
images, labels = dataiter.next()

def imshowo(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# print images
imshowo(torchvision.utils.make_grid(images))


outputs = model(images.view(-1, img_size*img_size*3).to(device))
rep = torch.Tensor.cpu(outputs).detach()
imshowo(torchvision.utils.make_grid(rep.view(test_batch,3,img_size,img_size)))